In [ ]:
import torch
import matplotlib.pyplot as plt
# local
import sys
if '..' not in sys.path:
    sys.path.insert(0, '..')

from datasets.import_dataset import import_dataset
from trainer import Trainer
import clamiter as ci
from transformation import RealNVP, train_prior, get_cov
from utils.plotting import *
import datasets.simulations as sim

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(f'device = {device}')


%load_ext autoreload
%autoreload 2

# Exotic Shapes


##  Pseudo Metric Space

In [ ]:

#* NEXT ARE DIFFERENT DEFINITIONS FOR 2 COMMUNITY GRAPHS
num_samples = 1000

# SHAPES FROM NF CLASS
# todo: import dataset 
#* two moons
graph_two_moons, dist_two_moons = sim.sample_normflows_dist(num_samples, 'TwoMoons', lorenz=True)

# #* circular gaussian mixture
graph_circ_gaus, dist_circ_gaus = sim.sample_normflows_dist(num_samples, 'Circ', lorenz=True)



_, axes_moons = plt.subplots(1, 2, figsize=(6, 3))
plot_2dgraph(graph_two_moons, lorenz_fig_lims=True, ax=axes_moons[0])
# plot_normflows_dist(dist_two_moons,shift=-0.5, scale=5, device=device, x_fig_lim=[-0.1, 1.1], ax=axes_moons[0])
plot_normflows_dist(dist_two_moons, lorenz=True, x_fig_lim=[0, 2.7], y_fig_lim=[-1.7,1.7], ax=axes_moons[1])

_, axes_circ = plt.subplots(1, 2, figsize=(6, 3))
plot_2dgraph(graph_circ_gaus, lorenz_fig_lims=True, ax=axes_circ[0])
# plot_normflows_dist(dist_circ_gaus,shift=-0.5, scale=5, device=device, x_fig_lim=[-0.1, 1.1], ax=axes_circ[0])
plot_normflows_dist(dist_circ_gaus,lorenz=True, x_fig_lim=[0, 2.7],y_fig_lim=[-1.7,1.7], ax=axes_circ[1])
#TODO: plot_normflow_dist needs to plot lorenz or not lorenz!!!!







# _, axes_chubs = plt.subplots(1, 2, figsize=(6, 3))
# plot_2dgraph(graph_two_chubs, lorenz_fig_lims=False, x_fig_lim=[-0.1, 1.1], ax=axes_chubs[0])
# plot_normflows_dist(dist_two_chubs,shift=-0.5, scale=5, device=device, x_fig_lim=[-0.1, 1.1], ax=axes_chubs[0])
# plot_normflows_dist(dist_two_chubs,shift=-0.5, scale=5, device=device, x_fig_lim=[-0.1, 1.1], ax=axes_chubs[1])


#todo: make sure that the distance runs. then put in a different folder and rearange



### IECLAM

In [ ]:
trainer_iegam_twomoons = Trainer(
                model_name='iegam',
                task=None,
                device=device,
                dataset=graph_two_moons.clone()
)

In [ ]:
# i want to optimize the trainer
losses_moons_ieclam, test_acc_moons_ieclam, val_acc_moons_ieclam = trainer_iegam_twomoons.train(
    init_feats=True,
    init_type='small_gaus',
    plot_every=-1,
    verbose=False
)

### PieClam

#### Two Moons

In [ ]:
config_triplets = [
                    ['feat_opt', 'n_iter', 2000],
                    ['feat_opt', 'lr', 0.00005],
                    ['prior_opt', 'n_iter', 1500],
                    ['prior_opt', 'lr', 0.0000005],
                    ['prior_opt', 'noise_amp', 0.1],
                    ['back_forth','n_back_forth', 50],
                    ['back_forth', 'first_func_in_fit', 'feat_opt']
                ]



trainer_pieclam_twomoons = Trainer(
                model_name='piegam',
                task=None,
                device=device,
                config_triplets_to_change=config_triplets,
                dataset=graph_two_moons.clone()
)

In [ ]:
# i want to optimize the trainer
losses_pieclam_moons, test_acc_moons, val_acc_moons = trainer_pieclam_twomoons.train(
    task_params={'d' : 0.2},
    plot_every=-1,
    init_feats=True,
    init_type='small_gaus',
    verbose=False,
    verbose_in_funcs=False
)
#todo: need to make default values for optimization and that the individual configurations be only some of the values...

# make default optimization values. use the mighty config and the individual configs should be config triplets

In [ ]:
plt.plot(get_prob_graph(trainer_bigclam_distance_twomoons.data.x, lorenz=False).cpu().flatten(), linestyle=None)


In [ ]:
fig, axes = plt.subplots(1, 3, figsize=(15, 5))
axes[0].plot(losses_moons)
axes[0].set_title('loss')
axes[1].plot(logcut_moons)
axes[1].set_title('log cut')
axes[2].plot(l2_moons)
axes[2].set_title('l2')


In [ ]:

#todo: make optimization for d that provides the minimum for log cut without the d

#todo: add the option to measure with sbm in train function add an attribute in the data object of the sbm that is the edge attr for the sbm
#First
# todo: recreate priors with pclam
# todo: make graphs in timecone and recreate with pieclam

# sbm
# todo: sbm and measure log cut in train. add also cut distance to test accuracy
# todo: the two other sbms ron suggested
# todo: 2,3,4 communities in pclam for the sbm and see that 2 is the best
# todo: 2,4 communities in pieclam for the sbm and show that 4 is the best 


#### Circle

In [ ]:
config_triplets = [
#                     ['feat_opt', 'n_iter', 2000],
#                     ['feat_opt', 'lr', 0.00005],
#                     ['prior_opt', 'n_iter', 1500],
#                     ['prior_opt', 'lr', 0.0000005],
#                     ['prior_opt', 'noise_amp', 0.25],
#                     ['back_forth','n_back_forth', 50],
#                     ['back_forth', 'first_func_in_fit', 'feat_opt']
                ]



trainer_iegam_circ = Trainer(
                model_name='iegam',
                task=None,
                device=device,
                config_triplets_to_change=config_triplets,
                dataset=graph_circ_gaus.clone()
)

losses_iegam_circ, acc_test_iegam_circ, acc_val_iegam_circ = trainer_iegam_circ.train(
    task_params={'d' : 0.2},
    plot_every=-1,
    init_feats=True,
    init_type='small_gaus',
    verbose=False,
    verbose_in_funcs=False
)

In [ ]:
# CIRC
config_triplets = [
                    ['feat_opt', 'n_iter', 2000],
                    ['feat_opt', 'lr', 0.00005],
                    ['prior_opt', 'n_iter', 1500],
                    ['prior_opt', 'lr', 0.0000005],
                    ['prior_opt', 'noise_amp', 0.1],
                    ['back_forth','n_back_forth', 50],
                    ['back_forth', 'first_func_in_fit', 'feat_opt']
                ]



trainer_piegam_circ = Trainer(
                model_name='piegam',
                task=None,
                device=device,
                config_triplets_to_change=config_triplets,
                dataset=graph_circ_gaus.clone()
)

In [ ]:
# i want to optimize the trainer
losses_piegam_circ, acc_test_piegam_circ, acc_val_piegam_circ = trainer_piegam_circ.train(
    task_params={'d' : 0.2},
    plot_every=5,
    init_feats=True,
    init_type='small_gaus',
    verbose=False,
    verbose_in_funcs=False
)
#todo: need to make default values for optimization and that the individual configurations be only some of the values...

# make default optimization values. use the mighty config and the individual configs should be config triplets